<a href="https://colab.research.google.com/github/Jeevesh8/arg_mining/blob/main/experiments/long_context_am.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Dependencies

In [ ]:
%%capture
#if running on colab, install below 4
#!git clone https://github.com/Jeevesh8/arg_mining
#!pip install transformers
#!pip install seqeval datasets allennlp
#!pip install flax

#if connected to local runtime, run the next command too
#pip install bs4 tensorflow torch 



*   Update ``arg_mining/datasets/cmv_modes/configs.py`` as per your requirements.



In [ ]:
#Run to ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Load Metric

In [ ]:
from datasets import load_metric
metric = load_metric('seqeval')

### Define & Load Tokenizer, Model, Dataset

In [ ]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
model_version = 'allenai/longformer-base-4096'

In [ ]:
%%capture
from transformers import LongformerTokenizer, AutoModel
tokenizer = LongformerTokenizer.from_pretrained(model_version)
transformer_model = AutoModel.from_pretrained(model_version).to(device)

In [ ]:
%%capture
from arg_mining.datasets.cmv_modes import load_dataset, data_config

tokenizer.add_tokens(data_config["special_tokens"])

transformer_model.resize_token_embeddings(len(tokenizer))

def get_datasets():
    train_dataset, valid_dataset, test_dataset = load_dataset(tokenizer=tokenizer,
                                                              train_sz=80,
                                                              test_sz=20,)
    return train_dataset, valid_dataset, test_dataset

### Define layers for a Linear-Chain-CRF

In [ ]:
import torch.nn as nn

from allennlp.modules.conditional_random_field import ConditionalRandomField as crf

ac_dict = data_config["arg_components"]

allowed_transitions =([(ac_dict["B-C"], ac_dict["I-C"]), 
                       (ac_dict["B-P"], ac_dict["I-P"])] + 
                      [(ac_dict["I-C"], ac_dict[ct]) 
                        for ct in ["I-C", "B-C", "B-P", "O"]] +
                      [(ac_dict["I-P"], ac_dict[ct]) 
                        for ct in ["I-P", "B-C", "B-P", "O"]] +
                      [(ac_dict["O"], ac_dict[ct]) 
                        for ct in ["O", "B-C", "B-P"]])
                    
linear_layer = nn.Linear(transformer_model.config.hidden_size,
                         len(ac_dict)).to(device)

crf_layer = crf(num_tags=len(ac_dict),
                constraints=allowed_transitions,
                include_start_end_transitions=False).to(device)

cross_entropy_layer = nn.CrossEntropyLoss(weight=torch.log(torch.tensor([3.8087, 54.6686, 3.4161, 45.3662, 2.4729], 
                                                                        device=device)), reduction='none')

### Global Attention Mask Utility for Longformer

In [ ]:
import numpy as np
from threading import Lock
MUTEX = Lock()

def get_global_attention_mask(tokenized_threads: np.ndarray) -> np.ndarray:
    """Returns an attention mask, with 1 where there are [USER{i}] tokens and 
    0 elsewhere.
    """
    mask = np.zeros_like(tokenized_threads)
    for user_token in ["UNU"]+[f"[USER{i}]" for i in range(data_config["max_users"])]:
        MUTEX.acquire()
        try:
            user_token_id = tokenizer.encode(user_token)[1:-1]
        finally:
            MUTEX.release()
        mask = np.where(tokenized_threads==user_token_id, 1, mask)
    return np.array(mask, dtype=bool)

### Loss and Prediction Function

In [ ]:
from typing import Tuple

In [ ]:
def compute(batch: Tuple[torch.Tensor, torch.Tensor, torch.Tensor],
            preds: bool=False, cross_entropy: bool=True):
    """
    Args:
        batch:  A tuple having tokenized thread of shape [batch_size, seq_len],
                component type labels of shape [batch_size, seq_len], and a global
                attention mask for Longformer, of the same shape.
        
        preds:  If True, returns a List(of batch_size size) of Tuples of form 
                (tag_sequence, viterbi_score) where the tag_sequence is the 
                viterbi-decoded sequence, for the corresponding sample in the batch.
        
        cross_entropy:  This argument will only be used if preds=False, i.e., if 
                        loss is being calculated. If True, then cross entropy loss
                        will also be added to the output loss.
    
    Returns:
        Either the predicted sequences with their scores for each element in the batch
        (if preds is True), or the loss value summed over all elements of the batch
        (if preds is False).
    """
    tokenized_threads, comp_type_labels, global_attention_mask = batch
    
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    
    logits = linear_layer(transformer_model(input_ids=tokenized_threads,
                               attention_mask=pad_mask,
                               global_attention_mask=global_attention_mask).last_hidden_state)
    
    if preds:
        return crf_layer.viterbi_tags(logits, pad_mask)
    
    log_likelihood = crf_layer(logits, comp_type_labels, pad_mask)
    
    if cross_entropy:
        logits = logits.reshape(-1, logits.shape[-1])
        
        pad_mask, comp_type_labels = pad_mask.reshape(-1), comp_type_labels.reshape(-1)
        
        ce_loss = torch.sum(pad_mask*cross_entropy_layer(logits, comp_type_labels))
        
        return ce_loss - log_likelihood

    return -log_likelihood

### Define optimizer

In [ ]:
from itertools import chain

import torch.optim as optim

optimizer = optim.Adam(params = chain(transformer_model.parameters(),
                                      linear_layer.parameters(),
                                      crf_layer.parameters()),
                       lr = 2e-5)

### Training And Evaluation Loops

In [ ]:
def train(dataset):
    accumulate_over = 4
    
    optimizer.zero_grad()

    for i, (tokenized_threads, masked_threads, comp_type_labels, _ ) in enumerate(dataset):
        global_attention_mask = torch.tensor(get_global_attention_mask(tokenized_threads), 
                                             device=device, dtype=torch.int32)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0), 
                                        device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0), 
                                     device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0), 
                                        device=device, dtype=torch.long)

        loss = compute((tokenized_threads, 
                        comp_type_labels, 
                        global_attention_mask))/data_config["batch_size"]
        
        print("Loss: ", loss)
        loss.backward()
        
        if i%accumulate_over==accumulate_over-1:
            optimizer.step()
            optimizer.zero_grad()
    
    optimizer.step()

In [ ]:
def evaluate(dataset, metric):
    
    int_to_labels = {v:k for k, v in ac_dict.items()}
    
    for tokenized_threads, masked_threads, comp_type_labels, _ in dataset:
        
        global_attention_mask = torch.tensor(get_global_attention_mask(tokenized_threads), 
                                             device=device)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0), 
                                        device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0), 
                                     device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0), 
                                        device=device)

        preds = compute((tokenized_threads,
                         comp_type_labels,
                         global_attention_mask),
                        preds=True)
        
        lengths = torch.sum(torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0), 
                            axis=-1)
        
        preds = [ [int_to_labels[pred] for pred in pred[0][:lengths[i]]]
                  for i, pred in enumerate(preds)
                ]
        
        refs = [ [int_to_labels[ref] for ref in labels[:lengths[i]]]
                 for i, labels in enumerate(comp_type_labels.cpu().tolist())
               ]
        
        metric.add_batch(predictions=preds, 
                         references=refs)
    
    print(metric.compute())

### Final Training

In [ ]:
n_epochs = 40

In [ ]:
for epoch in range(n_epochs):
    print(f"------------EPOCH {epoch+1}---------------")
    train_dataset, _, test_dataset = get_datasets()
    train(train_dataset)
    evaluate(test_dataset, metric)

------------EPOCH 1---------------


fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.
2021-08-24 15:59:48.011456: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-08-24 15:59:48.012489: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Loss:  tensor(3422.4038, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2775.5894, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3633.9263, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3353.6274, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2218.7983, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2212.5637, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2734.7212, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2890.7068, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1855.8928, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2002.3740, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2185.3320, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3409.1895, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2746.1924, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3949.2393, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(4037.3809, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  ten

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(2693.9438, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2062.6953, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2848.0596, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2348.4565, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1531.7446, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1564.5149, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1921.3455, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2109.8774, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1481.2172, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1657.8213, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1839.0326, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2702.5090, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2247.9382, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3271.6003, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3447.1890, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  ten

...

Loss:  tensor(14.9811, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.4420, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.4197, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(4.9325, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.0586, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.6535, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.7686, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(249.8771, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.5538, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.8290, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(10.4971, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(8.2284, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(436.3860, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1071.0330, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1173.4229, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(13.4275, device='cuda:0', gra

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(14.5853, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.1347, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.1567, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(4.6469, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(5.8330, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.4436, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.5231, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(241.6990, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.2847, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.6001, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(10.3659, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.8619, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(389.9162, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1111.0632, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1138.3104, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(13.1872, device='cuda:0', gra

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(14.2987, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.0120, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(5.9605, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(4.4551, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(5.6535, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.3932, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.3486, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(234.8909, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.0872, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(6.2045, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(10.0982, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(7.3839, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(410.2209, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1046.7727, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1117.1206, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(12.7867, device='cuda:0', gra